In [1]:
import helper_functions
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
lst = helper_functions.mhealth_get_dataset()
subjects = pd.DataFrame(lst)

In [ ]:
def get_subject_df(df, idx):
    return pd.DataFrame(subjects.data[idx])

In [5]:
sub = []
for i in range(0, 10):
    sub.append(get_subject_df(subjects, i))


In [6]:
# sub[0][sub[0][23] != 0]

In [7]:
from helper_functions import mhealth_get_dataset
import random
import tensorflow as tf
import numpy as np
from collections import defaultdict

# Hyperparameters
# This is how many samples per col
data_length = 2

# Prep data


# Get the dataset
dataset=mhealth_get_dataset()

# shuffle dataset
random.shuffle(dataset)

# get 8 training users and 2 test users
training_users,test_data = dataset[:6], dataset[6:]

    

In [8]:
# for lst in training_users:
#     print(lst.keys())

In [9]:
# training_users[0]['data'][0][23]

In [10]:
# # for user in training_users:
# for user_data in training_users[0]['data']:
#     #Removing records of no activity
#     if user_data[23] == 0:
#         training_users[0]['data'].remove(user_data)


In [33]:
for user in training_users:
    user['data'] = [lst for lst in user['data'] if lst[23] != 0]

In [34]:
for user in test_data:
    user['data'] = [lst for lst in user['data'] if lst[23] != 0]

In [15]:
# import pandas as pd
# tr_df = pd.DataFrame(training_users)

In [25]:
# tr_data_df = pd.DataFrame(tr_df['data'][0])

In [31]:
# tr_data_df[23].describe()

In [35]:

training_data_pre  = defaultdict(list) 
test_data_pre =defaultdict(list)

counter=0
previous_label=None
to_put=[]
for user in training_users:
    for user_data in user['data']:
        if not previous_label==user_data[23]:
            counter=1
            to_put=[]
            previous_label=user_data[23]
            user_data.pop()
            to_put+=user_data
        elif previous_label==user_data[23]:
            counter+=1
            user_data.pop()
            to_put+=user_data
        if counter == data_length:
            training_data_pre[previous_label].append(to_put)
            to_put=[]

counter=0
previous_label=None
to_put=[]
for user in test_data:
    for user_data in user['data']:
        if not previous_label==user_data[23]:
            counter=1
            to_put=[]
            previous_label=user_data[23]
            user_data.pop()
            to_put+=user_data
        elif previous_label==user_data[23]:
            counter+=1
            user_data.pop()
            to_put+=user_data
        if counter == data_length:
            test_data_pre[previous_label].append(to_put)
            to_put=[]

training_data,training_labels=[],[]
test_data,test_labels=[],[]

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

for training_label,training in training_data_pre.items():
    for train in training:
        training_labels.append(int(training_label))
        training_data.append(train)

training_data=np.array(training_data)


for training_label,training in test_data_pre.items():
    for train in training:
        test_labels.append(int(training_label))
        test_data.append(train)

test_data=np.array(test_data)

training_labels_len=len(training_labels)
labels=training_labels+test_labels
labels=get_one_hot(np.array(labels),13)
training_labels=labels[:training_labels_len]
test_labels=labels[training_labels_len:]

# Okay lets create our model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(data_length*23), # Our input layer
    tf.keras.layers.Dense(500,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(500),
    tf.keras.layers.Dense(13,activation=tf.keras.activations.softmax) # Our output layer we have 13 classifcations

])
# Compile our model
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

model.fit(training_data,training_labels,validation_data=(test_data,test_labels),epochs=100)
# -9.2305,3.9479,-4.0236,0.10884,-0.058608,3.4836,-7.467,-8.0885,0.24675,-0.78799,-0.21807,0.19791,18.714,5.4366,2.2399,-5.5648,7.5238,0.47255,-0.25051,1.0172,9.3079,-2.787,-15.309,9
# -8.6677,3.5852,-4.5615,-0.054422,0.22606,-0.75304,-7.1983,-5.9716,0.24675,-0.78799,-0.21807,0.25394,7.7039,9.586,-0.92224,-5.7833,9.6692,0.47255,-0.25051,1.0172,4.6822,1.4321,-15.444,9
test_array = np.array([[-9.2305,3.9479,-4.0236,0.10884,-0.058608,3.4836,-7.467,-8.0885,0.24675,-0.78799,-0.21807,0.19791,18.714,5.4366,2.2399,-5.5648,7.5238,0.47255,-0.25051,1.0172,9.3079,-2.787,-15.309,-8.6677,3.5852,-4.5615,-0.054422,0.22606,-0.75304,-7.1983,-5.9716,0.24675,-0.78799,-0.21807,0.25394,7.7039,9.586,-0.92224,-5.7833,9.6692,0.47255,-0.25051,1.0172,4.6822,1.4321,-15.444]])

print(model.predict([test_array])[0][8]*100)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 71 samples, validate on 48 samples
Epoch 1/100
71/71 [==============================] - 0s 5ms/sample - loss: 13.7912 - acc: 0.1268 - val_loss: 30.7258 - val_acc: 0.1458
Epoch 2/100
71/71 [==============================] - 0s 1ms/sample - loss: 17.9408 - acc: 0.3239 - val_loss: 26.3206 - val_acc: 0.1458
Epoch 3/100
71/71 [==============================] - 0s 1ms/sample - loss: 11.7259 - acc: 0.4085 - val_loss: 22.1405 - val_acc: 0.2500
Epoch 4/100
71/71 [==============================] - 0s 845us/sample - loss: 4.8179 - acc: 0.4789 - val_loss: 27.0858 - val_acc: 0.3542
Epoch 5/100
71/71 [==============================] - 0s 1ms/sample - loss: 6.9782 - acc: 0.5352 - val_loss: 25.3570 - val_acc: 0.3333
Epoch 6/100
71/71 [==============================] - 0s 7ms/sample - loss: 4.3364 - acc: 0.5915 - val_loss: 24.6817 - val_acc: 0.3333
Epoch 7/100
71/71 [==========